In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
tf.__version__

'2.7.0'

In [47]:
def load_X(path):
    X_signals = []
    files = os.listdir(path)
    files.sort(key=str.lower)
    #file
    #['train_acc_x.txt', 'train_acc_y.txt', 'train_acc_z.txt', 'train_gyr_x.txt', 'train_gyr_y.txt', 'train_gyr_z.txt']
    for my_file in files:
        fileName = os.path.join(path,my_file)
        file = open(fileName, 'r')
        X_signals.append(
            [np.expand_dims(np.array(cell, dtype=np.float32),axis=1) for cell in [
                row.strip().split(' ') for row in file
            ]]
        )
        file.close()
        #X_signals = 6*totalStepNum*128
    return np.transpose(np.array(X_signals), (1, 2, 0))#(totalStepNum*128*6)


In [48]:
# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    y_ = y_ - 1
    #one_hot
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS


In [49]:
    train_data = load_X('D:/College/ANACONDA/Datasets/Dataset #1/train/Inertial Signals')
    X_test = load_X('D:/College/ANACONDA/Datasets/Dataset #1/test/Inertial Signals')
    train_label = load_y('D:/College/ANACONDA/Datasets/Dataset #1/train/y_train.txt')
    test_label = load_y('D:/College/ANACONDA/Datasets/Dataset #1/test/y_test.txt')
    print(len(train_data), "Training series")
    print(len(X_test), "testing series")


ValueError: axes don't match array

In [ ]:
    batch_size = 1500
    number_of_epochs = 200
    optimizer = Adam()
    verbosity_mode = 1
    validation_split = 0.20
    additional_metrics = ['accuracy']
    loss_function = BinaryCrossentropy()
    n_steps = len(train_data[0])
    print( n_steps)

In [50]:
    n_inputs = len(train_data[0][0])  # Features count is of 9: 3 * 3D sensors features over time
    n_hidden = 64  # nb of neurons inside the neural network
    n_classes = 118  # Final output classes

In [51]:
# Pad all sequences
padded_inputs = pad_sequences(train_data, maxlen=n_steps, value = 0.0) # 0.0 because it corresponds with <PAD>
padded_inputs_test = pad_sequences(X_test, maxlen=n_steps, value = 0.0) # 0.0 because it corresponds with <PAD>


In [52]:
# Define the Keras model
#model = Sequential()
#model.add(Embedding(len(train_data[0][0]), n_hidden ))
#model.add(LSTM(10))
#model.add(Dense(1, activation='relu'))

inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(len(train_data[0][0]), 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 128)         768       
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 198,529
Trainable params: 198,529
Non-trainab

In [53]:
# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)


In [54]:
# Give a summary
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 128)         768       
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 198,529
Trainable params: 198,529
Non-trainab

In [55]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [56]:
model = Sequential()
model.add(LSTM(10, input_shape=(128,6)))
model.add(Dense(118, activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(train_data, train_label,   batch_size=1500, epochs=2, validation_split= 0.20 )

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 10)                680       
                                                                 
 dense_10 (Dense)            (None, 118)               1298      
                                                                 
Total params: 1,978
Trainable params: 1,978
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
18/18 [==============================] - 5s 219ms/step - loss: 9.4001 - accuracy: 0.0086 - val_loss: 8.9187 - val_accuracy: 0.0174
Epoch 2/2
18/18 [==============================] - 3s 183ms/step - loss: 8.6910 - accuracy: 0.0262 - val_loss: 8.4511 - val_accuracy: 0.0403
